In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# data from CSV files
teams_df = pd.read_csv('teams.csv')
technical_officials_df = pd.read_csv('technical_officials.csv')
events_df = pd.read_csv('events.csv')
nocs_df = pd.read_csv('nocs.csv')
schedule_preliminary_df = pd.read_csv('schedules_preliminary.csv')
schedule_df = pd.read_csv('schedules.csv')
torch_route_df = pd.read_csv('torch_route.csv')
medals_total_df = pd.read_csv('medals_total.csv')
medals_df = pd.read_csv('medals.csv')
venues_df = pd.read_csv('venues.csv')
athletes_df = pd.read_csv('athletes.csv')
coaches_df = pd.read_csv('coaches.csv')
medallists_df = pd.read_csv('medallists.csv')

In [ ]:
# Cleaning up the 'disciplines' column in technical_officials.csv -> this is because disciplines in this table is a list, and rest of the tables have singular data in this type of column.
import ast
technical_officials_df['disciplines'] = technical_officials_df['disciplines'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
technical_officials_df = technical_officials_df.explode('disciplines')
technical_officials_df['disciplines'] = technical_officials_df['disciplines'].str.strip()

In [ ]:
# Cleaning up the 'events' column in athletes.csv -> this is because events in this table is a list, and rest of the tables have singular data in this type of column.
import ast
athletes_df['events'] = athletes_df['events'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x).head(1000)
athletes_df = athletes_df.explode('events')
athletes_df['events'] = athletes_df['events'].str.strip()

In [ ]:
#1 Athlete participation and medal-winning by event
athletes_medals_by_event = pd.merge(athletes_df, medals_df, on='name', how='inner')
athletes_performance = athletes_medals_by_event.groupby('discipline')['medal_type'].count().reset_index()
athletes_performance.columns = ['discipline', 'medal_count']

athletes_performance = athletes_performance.sort_values(by='medal_count', ascending=False).head(20)

plt.figure(figsize=(14, 8))
sns.barplot(x='medal_count', y='discipline', data=athletes_performance, palette='crest')
plt.title('Top 20 Events with Highest Medal Wins')
plt.xlabel('Number of Medals Won')
plt.ylabel('Sports Discipline')
plt.show()

In [ ]:
#2 Event distribution over days
schedule_df['day'] = pd.to_datetime(schedule_df['day'])
events_by_date = schedule_df.groupby('day').size().reset_index(name='event_count')

plt.figure(figsize=(14, 6))
sns.lineplot(x='day', y='event_count', data=events_by_date, marker='o')
plt.title('Number of Events Over Days')
plt.xlabel('Date')
plt.ylabel('Number of Events')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#3 Torch relay visualization (geographical distribution)
torch_route_df['city'] = torch_route_df['city'].str.strip()  # Clean city names
torch_route_count = torch_route_df['city'].value_counts().reset_index()
torch_route_count.columns = ['city', 'count']

plt.figure(figsize=(12, 6))
sns.barplot(x='count', y='city', data=torch_route_count.head(20), palette='viridis')
plt.title('Top 20 Cities in Torch Relay Route')
plt.xlabel('Number of Torch Visits')
plt.ylabel('City')
plt.show()

In [ ]:
#4 Merge teams with medals to find team performance
teams_performance = pd.merge(teams_df, medals_df, left_on='discipline', right_on='discipline')
teams_medals_count = teams_performance.groupby('team')['medal_type'].count().reset_index()
teams_medals_count = teams_medals_count.sort_values(by='medal_type', ascending=False).head(20)

plt.figure(figsize=(12, 6))
sns.barplot(x='medal_type', y='team', data=teams_medals_count, palette='Spectral')
plt.title('Top Performing Teams')
plt.xlabel('Number of Medals')
plt.ylabel('Team')
plt.show()

In [ ]:
#5 Medal win rate calculation
athlete_count_by_country = athletes_df.groupby('country')['name'].count().reset_index()
medals_by_country = medals_df.groupby('country')['medal_type'].count().reset_index()

win_rate_df = pd.merge(athlete_count_by_country, medals_by_country, on='country')
win_rate_df['win_rate'] = win_rate_df['medal_type'] / win_rate_df['name']
win_rate_df = win_rate_df.sort_values(by='win_rate', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='win_rate', y='country', data=win_rate_df.head(20), palette='plasma')
plt.title('Top 20 Countries by Medal Win Rate')
plt.xlabel('Win Rate')
plt.ylabel('Country')
plt.show()

In [ ]:
#6 Coach-to-athlete ratio
coach_count = coaches_df.groupby('country')['name'].count().reset_index()
athlete_count = athletes_df.groupby('country')['name'].count().reset_index()

ratio_df = pd.merge(coach_count, athlete_count, on='country', how='inner')
ratio_df.columns = ['country', 'coaches', 'athletes']
ratio_df['coach_to_athlete_ratio'] = ratio_df['coaches'] / ratio_df['athletes']

ratio_df = ratio_df.sort_values(by='coach_to_athlete_ratio', ascending=False).head(20)
sns.barplot(x='coach_to_athlete_ratio', y='country', data=ratio_df, palette='crest')
plt.title('Top 20 Countries by Coach-to-Athlete Ratio')
plt.xlabel('Coach-to-Athlete Ratio')
plt.ylabel('Country')
plt.show()

In [ ]:
#7 Medal heatmap by discipline and country
medal_heatmap_data = medals_df.groupby(['country', 'discipline'])['medal_type'].count().unstack(fill_value=0)

plt.figure(figsize=(14, 8))
sns.heatmap(medal_heatmap_data, cmap='YlGnBu', annot=False)
plt.title('Medal Distribution Heatmap by Discipline and Country')
plt.xlabel('Discipline')
plt.ylabel('Country')
plt.show()